
Alembic の初期設定を行い、最初のテーブルを作成するまでの手順

1. 各種ファイルを作成
    ```console
    $ alembic init migrations
    ```
2. `models.py` を作成する
3. `migrations/env.py` を設定する
    - `target_metadata` の値に `models.py` で作成したベースモデルを指定する
    - DB 接続情報を環境変数から取得できるようにする
4. マイグレーションファイルを生成する
    ```console
    $ alembic revision --autogenerate -m "Create tables" --rev-id $(date -u +"%Y%m%d_%H%M%S")
    ```
5. マイグレーションファイルを適用する
    ```console
    $ alembic upgrade head
    ```

In [5]:
from sqlalchemy.orm import Session
from sqlalchemy import select, text
from tables import UserTable, ArticleTable, CommentTable
from database import fetch_all, engine

In [6]:
# 接続確認
fetch_all(select(text("1")))

[1]

## データ追加

In [7]:
from datetime import datetime, timedelta
from utils import hash

with Session(engine) as session:
    # User
    users = [
        UserTable(
            email="alice@example.com",
            name="Alice",
            password_hash=hash("banana"),
        ),
        UserTable(
            email="bob@example.com",
            name="Bob",
            password_hash=hash("dog"),
        ),
        UserTable(
            email="carol@example.com",
            name="Carol",
            password_hash=hash("train"),
        ),
    ]
    session.add_all(users)
    session.flush()  # ID を確定させる
    alice, bob, carol = users

    # Article
    articles = [
        ArticleTable(
            author_id=alice.id,
            title="Hello, SQLAlchemy 2.x",
            body="This is a dummy article about SQLAlchemy 2.x typed ORM.",
            published_at=datetime.now() - timedelta(days=2),
        ),
        ArticleTable(
            author_id=alice.id,
            title="Alembic autogenerate tips",
            body="A few practical tips for Alembic migrations.",
            published_at=datetime.now() - timedelta(days=2),
        ),
        ArticleTable(
            author_id=bob.id,
            title="JSONB patterns in Postgres",
            body="Dummy content: indexing JSONB, querying, and constraints.",
            published_at=None,
        ),
    ]
    session.add_all(articles)
    session.flush()
    a1, a2, a3 = articles

    # Comment
    comments = [
        CommentTable(
            article_id=a1.id,
            author_id=bob.id,
            body="yayayay",
        ),
        CommentTable(
            article_id=a1.id,
            author_id=carol.id,
            body="wowowow",
        ),
    ]
    session.add_all(comments)

    session.commit()

## Pydantic モデルを自動生成

In [1]:
from olivier import sqlalchemy_model_to_pydantic_model_definition, create_partial
from tables import UserTable, ArticleTable, CommentTable


User = sqlalchemy_model_to_pydantic_model_definition(UserTable)
User = create_partial(User, exclude={"password_hash"})
User["config"]["extra"] = "ignore"

Article = sqlalchemy_model_to_pydantic_model_definition(ArticleTable)

Comment = sqlalchemy_model_to_pydantic_model_definition(CommentTable)

In [2]:
from olivier import build_model_definitions
from utils import ruff_format

with open("generated.py", "w") as f:
    code = build_model_definitions([User, Article, Comment])
    f.write(ruff_format(code, line_length=120))